In [1]:
import sys

import os
from snappy import GPF
from snappy import ProductIO
from snappy import HashMap
import jpy

#from snappy import jpy
#import matplotlib.pyplot as plt
#import matplotlib.colors as colors
#import numpy as np
import json
 # functions
# Hashmap is used to give us access to all JAVA operators
#HashMap = jpy.get_type('java.util.HashMap')
from glob import iglob
from os.path import join
from snappy import GPF, ProductIO, HashMap

parameters = HashMap()

In [2]:


# Initialize GPF with default parameters
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

# Define functions for each processing step
def read(filename):
    try:
        print(f'Reading {filename}...')
        return ProductIO.readProduct(filename)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

def topsar_split(product, IW, firstBurstIndex, lastBurstIndex):
    parameters = HashMap()
    parameters.put('subswath', IW)
    parameters.put('firstBurstIndex', firstBurstIndex)
    parameters.put('lastBurstIndex', lastBurstIndex)
    parameters.put('selectedPolarisations', 'VV')
    return GPF.createProduct("TOPSAR-Split", parameters, product)

def apply_orbit_file(product):
    parameters = HashMap()
    parameters.put("Orbit State Vectors", "Sentinel Precise (Auto Download)")
    parameters.put("Polynomial Degree", 3)
    parameters.put("Do not fail if new orbit file is not found", True)
    return GPF.createProduct("Apply-Orbit-File", parameters, product)

def back_geocoding(products):
    parameters = HashMap()
    parameters.put("Digital Elevation Model", "SRTM 1Sec HGT (Auto Download)")
    parameters.put("DEM Resampling Method", "BILINEAR_INTERPOLATION")
    parameters.put("Resampling Type", "BILINEAR_INTERPOLATION")
    parameters.put("Mask out areas with no elevation", True)
    parameters.put("Output Deramp and Demod Phase", True)
    parameters.put("Disable Reramp", True)
    return GPF.createProduct("Back-Geocoding", parameters, products)

def Enhanced_Spectral_Diversity(product):
    parameters = HashMap()
    parameters.put("fineWinWidthStr2",'512')
    parameters.put("fineWinHeightStr",'512')
    parameters.put("fineWinAccAzimuth",'16')
    parameters.put("fineWinAccRange",'16')
    parameters.put("fineWinOversampling",'128')
    parameters.put("xCorrThreshold",0.1)
    parameters.put("cohThreshold",0.3)
    parameters.put("numBlocksPerOverlap",10)
    parameters.put("esdEstimator",'Periodogram')
    parameters.put("weightFunc",'Inv Quadratic')
    parameters.put("temporalBaselineType",'Number of images')
    parameters.put("maxTemporalBaseline",4)
    parameters.put("integrationMethod",'L1 and L2')
    parameters.put("doNotWriteTargetBands",False)
    parameters.put("useSuppliedRangeShift",False)
    parameters.put("overallRangeShift",'0.0')
    parameters.put("useSuppliedAzimuthShift",False)
    parameters.put("overallAzimuthShift",'0.0')
    return GPF.createProduct("Enhanced-Spectral-Diversity", parameters, product)

def coherence(product):
    parameters = HashMap()
    parameters.put("Single Refrence", True)
    parameters.put("Square Pixel", True)
    parameters.put("Coherence Range Window Size", 20)
    parameters.put("Coherence Azimuth Window Size", 20)
    return GPF.createProduct("Coherence", parameters, product)

def topsar_deburst(product):
    parameters = HashMap()
    parameters.put("Polarisations", "VV")
#     parameters.put('ignoreNoDataValue', True)  # Instructs the mosaic to ignore no-data values
#     parameters.put('noDataValue', 0)  # Adjust this to match the no-data value in your images
    return GPF.createProduct("TOPSAR-Deburst", parameters, product)

def speckle_filter(product):
    parameters = HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 7)
    parameters.put('filterSizeY', 7)
    parameters.put('dampingFactor', 2)
    parameters.put('estimateENL', True)
    parameters.put('enl', '1.0')
    parameters.put('numLooksStr', '1')
    parameters.put('targetWindowSizeStr', '3x3')
#     parameters.put('sigmaStr', '0.9')
#     parameters.put('anSize', '50')

    return GPF.createProduct('Speckle-Filter', parameters,product)

def multilook(product, ML_nRgLooks):
    parameters = HashMap()
    parameters.put('grSquarePixel', True)
    parameters.put("nRgLooks", ML_nRgLooks)
    return GPF.createProduct("Multilook", parameters, product)

def goldstein_phase_filtering(product):
    parameters = HashMap()
    parameters.put("Adaptive Filter Exponent in[0,1]:", 1.0)
    parameters.put("FFT Size", 64)
    parameters.put("Window Size", 3)
    return GPF.createProduct("GoldsteinPhaseFiltering", parameters, product)

def set_no_data_values(product, no_data_value):
    # Directly access and set no-data values for each band
    Band = jpy.get_type('org.esa.snap.core.datamodel.Band')
    for band_name in product.getBandNames():
        band = product.getBand(band_name)
        if isinstance(band, Band):
            band.setNoDataValue(no_data_value)
            band.setNoDataValueUsed(True)

def terrain_correction(product):
    proj = '''PROJCS["WGS 84 / UTM zone 33N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",15],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32633"]]'''
    
    parameters = HashMap()
    parameters.put('demName', 'SRTM 3Sec')
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('pixelSpacingInMeter', 20.0)
    parameters.put('mapProjection', proj)
    parameters.put('nodataValueAtSea', False)
    parameters.put('maskOutAreasWithoutElevation', True)
    parameters.put('ignoreNoDataValue', True)  # Instructs the mosaic to ignore no-data values
#     parameters.put('noDataValue', 0)  # Adjust this to match the no-data value in your images
#     parameters.put('saveSelectedSourceBand', True)
    
    
#     parameters.put('sourceBands', 'coh_IW2_VV_09Jun2019_03Jun2019')
    return GPF.createProduct('Terrain-Correction', parameters, product)



# Get the Java ArrayList class
ArrayList = jpy.get_type('java.util.ArrayList')

# Adjust the mosaic_products function accordingly
def mosaic_products(file_paths, output_filename):
    products = ArrayList()
    for file_path in file_paths:
        product = ProductIO.readProduct(file_path)
        products.add(product)
    
    parameters = HashMap()
    parameters.put('resamplingType', 'BILINEAR_INTERPOLATION')
    parameters.put('variableDownsamplingMethod', 'Mean')
    parameters.put('pixelSizeX', '20')  # Adjust according to your needs
    parameters.put('pixelSizeY', '20')  # Adjust according to your needs
#     parameters.put('ignoreNoDataValue', True)  # Instructs the mosaic to ignore no-data values
#     parameters.put('noDataValue', 0)  # Adjust this to match the no-data value in your images

    mosaicked_product = GPF.createProduct("Mosaic", parameters, products)
    ProductIO.writeProduct(mosaicked_product, output_filename, 'GeoTIFF')

def write_product(product, filename, format='GEOTIFF'):
    ProductIO.writeProduct(product, filename, format)

# Example of how to use these functions in a pipeline
def process_insar_pipeline(filename_1, filename_2, IW, firstBurstIndex, lastBurstIndex, out_filename):
    product_1 = read(filename_1)
    product_2 = read(filename_2)

    if product_1 and product_2:
        topsar_1 = topsar_split(product_1, IW, firstBurstIndex, lastBurstIndex)
        topsar_2 = topsar_split(product_2, IW, firstBurstIndex, lastBurstIndex)
        orbit_1 = apply_orbit_file(topsar_1)
        orbit_2 = apply_orbit_file(topsar_2)
        back_geo = back_geocoding([orbit_1, orbit_2])
        ESD = Enhanced_Spectral_Diversity(back_geo)
        
        coherence_product = coherence(ESD)
        
        debursted = topsar_deburst(coherence_product)
        
#         multilooked = speckle_filter(debursted)  # Example value for ML_nRgLooks
#         filtered = goldstein_phase_filtering(multilooked)
        corrected = terrain_correction(debursted)

        # Correct place to call set_no_data_values
        set_no_data_values(corrected, 0)  # This modifies 'corrected' in-place, doesn't return a new product

        # Now pass 'corrected' to write_product, as 'corrected' is the final product you wish to write
        write_product(corrected, out_filename)
# product = apply_orbit_file(product)
# if product is None:
#     raise RuntimeError("apply_orbit_file failed to execute properly.")
        

In [5]:
# Main loop for processing across folders
base_product_path = 'F:/Coherence/Data/2019/Ascending'
folders = [f'frame_201906*_{i}_' for i in range(1, 12)]  # Adjust as per your folders
generated_files = []  # Collect paths of all generated images

processed_folder_path = os.path.join(base_product_path, 'processed_vv')
if not os.path.exists(processed_folder_path):
    os.makedirs(processed_folder_path)



subswaths = ['IW1', 'IW2', 'IW3']  # List of subswaths to process
firstBurstIndex = 1
lastBurstIndex = 10  # Adjust these as needed

for folder in folders:
    folder_path = os.path.join(base_product_path, folder)
    input_S1_files = sorted(list(iglob(os.path.join(folder_path, '**', '*S1A*.zip'), recursive=True)))
    
    if len(input_S1_files) >= 2:
        for i in range(0, len(input_S1_files), 2):
            pre_landslide_s1 = input_S1_files[i]
            post_landslide_s1 = input_S1_files[i + 1] if i + 1 < len(input_S1_files) else None
            
            if pre_landslide_s1 and post_landslide_s1:
                for IW in subswaths:
                    base_filename = os.path.basename(pre_landslide_s1)
                    # Modify here to save outputs in the 'processed' folder
                    out_filename = os.path.join(processed_folder_path, f"{os.path.splitext(base_filename)[0]}_{IW}_coh.tif")
                    
                    print(f"Processing: {pre_landslide_s1}, {post_landslide_s1} for {IW}")
                    print(f"Output will be saved as: {out_filename}")
                    
                    process_insar_pipeline(pre_landslide_s1, post_landslide_s1, IW, firstBurstIndex, lastBurstIndex, out_filename)
    else:
        print(f"No sufficient images in {folder_path} for processing.")


Processing: F:/Coherence/Data/2019/Ascending\frame_20190603_1_\S1A_IW_SLC__1SDV_20190603T165053_20190603T165121_027518_031AEC_3ADD.zip, F:/Coherence/Data/2019/Ascending\frame_20190603_1_\S1A_IW_SLC__1SDV_20190615T165053_20190615T165121_027693_032036_8012.zip for IW1
Output will be saved as: F:/Coherence/Data/2019/Ascending\processed_vv\S1A_IW_SLC__1SDV_20190603T165053_20190603T165121_027518_031AEC_3ADD_IW1_coh.tif
Reading F:/Coherence/Data/2019/Ascending\frame_20190603_1_\S1A_IW_SLC__1SDV_20190603T165053_20190603T165121_027518_031AEC_3ADD.zip...
Reading F:/Coherence/Data/2019/Ascending\frame_20190603_1_\S1A_IW_SLC__1SDV_20190615T165053_20190615T165121_027693_032036_8012.zip...
Processing: F:/Coherence/Data/2019/Ascending\frame_20190603_1_\S1A_IW_SLC__1SDV_20190603T165053_20190603T165121_027518_031AEC_3ADD.zip, F:/Coherence/Data/2019/Ascending\frame_20190603_1_\S1A_IW_SLC__1SDV_20190615T165053_20190615T165121_027693_032036_8012.zip for IW2
Output will be saved as: F:/Coherence/Data/2019

Processing: F:/Coherence/Data/2019/Ascending\frame_20190603_2_\S1A_IW_SLC__1SDV_20190603T165119_20190603T165146_027518_031AEC_45B7.zip, F:/Coherence/Data/2019/Ascending\frame_20190603_2_\S1A_IW_SLC__1SDV_20190615T165119_20190615T165146_027693_032036_DFD9.zip for IW1
Output will be saved as: F:/Coherence/Data/2019/Ascending\processed_vv\S1A_IW_SLC__1SDV_20190603T165119_20190603T165146_027518_031AEC_45B7_IW1_coh.tif
Reading F:/Coherence/Data/2019/Ascending\frame_20190603_2_\S1A_IW_SLC__1SDV_20190603T165119_20190603T165146_027518_031AEC_45B7.zip...
Reading F:/Coherence/Data/2019/Ascending\frame_20190603_2_\S1A_IW_SLC__1SDV_20190615T165119_20190615T165146_027693_032036_DFD9.zip...
Processing: F:/Coherence/Data/2019/Ascending\frame_20190603_2_\S1A_IW_SLC__1SDV_20190603T165119_20190603T165146_027518_031AEC_45B7.zip, F:/Coherence/Data/2019/Ascending\frame_20190603_2_\S1A_IW_SLC__1SDV_20190615T165119_20190615T165146_027693_032036_DFD9.zip for IW2
Output will be saved as: F:/Coherence/Data/2019

Processing: F:/Coherence/Data/2019/Ascending\frame_20190603_3_\S1A_IW_SLC__1SDV_20190603T165143_20190603T165210_027518_031AEC_7A4C.zip, F:/Coherence/Data/2019/Ascending\frame_20190603_3_\S1A_IW_SLC__1SDV_20190615T165144_20190615T165211_027693_032036_E71E.zip for IW1
Output will be saved as: F:/Coherence/Data/2019/Ascending\processed_vv\S1A_IW_SLC__1SDV_20190603T165143_20190603T165210_027518_031AEC_7A4C_IW1_coh.tif
Reading F:/Coherence/Data/2019/Ascending\frame_20190603_3_\S1A_IW_SLC__1SDV_20190603T165143_20190603T165210_027518_031AEC_7A4C.zip...
Reading F:/Coherence/Data/2019/Ascending\frame_20190603_3_\S1A_IW_SLC__1SDV_20190615T165144_20190615T165211_027693_032036_E71E.zip...
Processing: F:/Coherence/Data/2019/Ascending\frame_20190603_3_\S1A_IW_SLC__1SDV_20190603T165143_20190603T165210_027518_031AEC_7A4C.zip, F:/Coherence/Data/2019/Ascending\frame_20190603_3_\S1A_IW_SLC__1SDV_20190615T165144_20190615T165211_027693_032036_E71E.zip for IW2
Output will be saved as: F:/Coherence/Data/2019

In [9]:


# Initialize GPF with default parameters
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

# Define functions for each processing step
def read(filename):
    try:
        print(f'Reading {filename}...')
        return ProductIO.readProduct(filename)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

def topsar_split(product, IW, firstBurstIndex, lastBurstIndex):
    parameters = HashMap()
    parameters.put('subswath', IW)
    parameters.put('firstBurstIndex', firstBurstIndex)
    parameters.put('lastBurstIndex', lastBurstIndex)
    parameters.put('selectedPolarisations', 'VH')
    return GPF.createProduct("TOPSAR-Split", parameters, product)

def apply_orbit_file(product):
    parameters = HashMap()
    parameters.put("Orbit State Vectors", "Sentinel Precise (Auto Download)")
    parameters.put("Polynomial Degree", 3)
    parameters.put("Do not fail if new orbit file is not found", True)
    return GPF.createProduct("Apply-Orbit-File", parameters, product)

def back_geocoding(products):
    parameters = HashMap()
    parameters.put("Digital Elevation Model", "SRTM 1Sec HGT (Auto Download)")
    parameters.put("DEM Resampling Method", "BILINEAR_INTERPOLATION")
    parameters.put("Resampling Type", "BILINEAR_INTERPOLATION")
    parameters.put("Mask out areas with no elevation", True)
    parameters.put("Output Deramp and Demod Phase", True)
    parameters.put("Disable Reramp", True)
    return GPF.createProduct("Back-Geocoding", parameters, products)

def Enhanced_Spectral_Diversity(product):
    parameters = HashMap()
    parameters.put("fineWinWidthStr2",'512')
    parameters.put("fineWinHeightStr",'512')
    parameters.put("fineWinAccAzimuth",'16')
    parameters.put("fineWinAccRange",'16')
    parameters.put("fineWinOversampling",'128')
    parameters.put("xCorrThreshold",0.1)
    parameters.put("cohThreshold",0.3)
    parameters.put("numBlocksPerOverlap",10)
    parameters.put("esdEstimator",'Periodogram')
    parameters.put("weightFunc",'Inv Quadratic')
    parameters.put("temporalBaselineType",'Number of images')
    parameters.put("maxTemporalBaseline",4)
    parameters.put("integrationMethod",'L1 and L2')
    parameters.put("doNotWriteTargetBands",False)
    parameters.put("useSuppliedRangeShift",False)
    parameters.put("overallRangeShift",'0.0')
    parameters.put("useSuppliedAzimuthShift",False)
    parameters.put("overallAzimuthShift",'0.0')
    return GPF.createProduct("Enhanced-Spectral-Diversity", parameters, product)

def coherence(product):
    parameters = HashMap()
    parameters.put("Single Refrence", True)
    parameters.put("Square Pixel", True)
    parameters.put("Coherence Range Window Size", 20)
    parameters.put("Coherence Azimuth Window Size", 20)
    return GPF.createProduct("Coherence", parameters, product)

def topsar_deburst(product):
    parameters = HashMap()
    parameters.put("Polarisations", "VH")
#     parameters.put('ignoreNoDataValue', True)  # Instructs the mosaic to ignore no-data values
#     parameters.put('noDataValue', 0)  # Adjust this to match the no-data value in your images
    return GPF.createProduct("TOPSAR-Deburst", parameters, product)

def speckle_filter(product):
    parameters = HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 7)
    parameters.put('filterSizeY', 7)
    parameters.put('dampingFactor', 2)
    parameters.put('estimateENL', True)
    parameters.put('enl', '1.0')
    parameters.put('numLooksStr', '1')
    parameters.put('targetWindowSizeStr', '3x3')
#     parameters.put('sigmaStr', '0.9')
#     parameters.put('anSize', '50')

    return GPF.createProduct('Speckle-Filter', parameters,product)

def multilook(product, ML_nRgLooks):
    parameters = HashMap()
    parameters.put('grSquarePixel', True)
    parameters.put("nRgLooks", ML_nRgLooks)
    return GPF.createProduct("Multilook", parameters, product)

def goldstein_phase_filtering(product):
    parameters = HashMap()
    parameters.put("Adaptive Filter Exponent in[0,1]:", 1.0)
    parameters.put("FFT Size", 64)
    parameters.put("Window Size", 3)
    return GPF.createProduct("GoldsteinPhaseFiltering", parameters, product)

def set_no_data_values(product, no_data_value):
    # Directly access and set no-data values for each band
    Band = jpy.get_type('org.esa.snap.core.datamodel.Band')
    for band_name in product.getBandNames():
        band = product.getBand(band_name)
        if isinstance(band, Band):
            band.setNoDataValue(no_data_value)
            band.setNoDataValueUsed(True)

def terrain_correction(product):
    proj = '''PROJCS["WGS 84 / UTM zone 33N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",15],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32633"]]'''
    
    parameters = HashMap()
    parameters.put('demName', 'SRTM 3Sec')
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('pixelSpacingInMeter', 20.0)
    parameters.put('mapProjection', proj)
    parameters.put('nodataValueAtSea', False)
    parameters.put('maskOutAreasWithoutElevation', True)
    parameters.put('ignoreNoDataValue', True)  # Instructs the mosaic to ignore no-data values
#     parameters.put('noDataValue', 0)  # Adjust this to match the no-data value in your images
#     parameters.put('saveSelectedSourceBand', True)
    
    
#     parameters.put('sourceBands', 'coh_IW2_VV_09Jun2019_03Jun2019')
    return GPF.createProduct('Terrain-Correction', parameters, product)



# Get the Java ArrayList class
ArrayList = jpy.get_type('java.util.ArrayList')

# Adjust the mosaic_products function accordingly
def mosaic_products(file_paths, output_filename):
    products = ArrayList()
    for file_path in file_paths:
        product = ProductIO.readProduct(file_path)
        products.add(product)
    
    parameters = HashMap()
    parameters.put('resamplingType', 'BILINEAR_INTERPOLATION')
    parameters.put('variableDownsamplingMethod', 'Mean')
    parameters.put('pixelSizeX', '20')  # Adjust according to your needs
    parameters.put('pixelSizeY', '20')  # Adjust according to your needs
#     parameters.put('ignoreNoDataValue', True)  # Instructs the mosaic to ignore no-data values
#     parameters.put('noDataValue', 0)  # Adjust this to match the no-data value in your images

    mosaicked_product = GPF.createProduct("Mosaic", parameters, products)
    ProductIO.writeProduct(mosaicked_product, output_filename, 'GeoTIFF')

def write_product(product, filename, format='GEOTIFF'):
    ProductIO.writeProduct(product, filename, format)

# Example of how to use these functions in a pipeline
def process_insar_pipeline(filename_1, filename_2, IW, firstBurstIndex, lastBurstIndex, out_filename):
    product_1 = read(filename_1)
    product_2 = read(filename_2)

    if product_1 and product_2:
        topsar_1 = topsar_split(product_1, IW, firstBurstIndex, lastBurstIndex)
        topsar_2 = topsar_split(product_2, IW, firstBurstIndex, lastBurstIndex)
        orbit_1 = apply_orbit_file(topsar_1)
        orbit_2 = apply_orbit_file(topsar_2)
        back_geo = back_geocoding([orbit_1, orbit_2])
        ESD = Enhanced_Spectral_Diversity(back_geo)
        
        coherence_product = coherence(ESD)
        
        debursted = topsar_deburst(coherence_product)
        
#         multilooked = speckle_filter(debursted)  # Example value for ML_nRgLooks
#         filtered = goldstein_phase_filtering(multilooked)
        corrected = terrain_correction(debursted)

        # Correct place to call set_no_data_values
        set_no_data_values(corrected, 0)  # This modifies 'corrected' in-place, doesn't return a new product

        # Now pass 'corrected' to write_product, as 'corrected' is the final product you wish to write
        write_product(corrected, out_filename)
# product = apply_orbit_file(product)
# if product is None:
#     raise RuntimeError("apply_orbit_file failed to execute properly.")
        

In [10]:
# Main loop for processing across folders
base_product_path = 'C:/Users/Prosper/Desktop/Data/2019'
folders = [f'2019_{i}' for i in range(1, 11)]  # Adjust as per your folders
generated_files = []  # Collect paths of all generated images

processed_folder_path = os.path.join(base_product_path, 'processed_vh')
if not os.path.exists(processed_folder_path):
    os.makedirs(processed_folder_path)



subswaths = ['IW1', 'IW2', 'IW3']  # List of subswaths to process
firstBurstIndex = 1
lastBurstIndex = 10  # Adjust these as needed

for folder in folders:
    folder_path = os.path.join(base_product_path, folder)
    input_S1_files = sorted(list(iglob(os.path.join(folder_path, '**', '*S1A*.zip'), recursive=True)))
    
    if len(input_S1_files) >= 2:
        for i in range(0, len(input_S1_files), 2):
            pre_landslide_s1 = input_S1_files[i]
            post_landslide_s1 = input_S1_files[i + 1] if i + 1 < len(input_S1_files) else None
            
            if pre_landslide_s1 and post_landslide_s1:
                for IW in subswaths:
                    base_filename = os.path.basename(pre_landslide_s1)
                    # Modify here to save outputs in the 'processed' folder
                    out_filename = os.path.join(processed_folder_path, f"{os.path.splitext(base_filename)[0]}_{IW}_coh.tif")
                    
                    print(f"Processing: {pre_landslide_s1}, {post_landslide_s1} for {IW}")
                    print(f"Output will be saved as: {out_filename}")
                    
                    process_insar_pipeline(pre_landslide_s1, post_landslide_s1, IW, firstBurstIndex, lastBurstIndex, out_filename)
    else:
        print(f"No sufficient images in {folder_path} for processing.")


Processing: C:/Users/Prosper/Desktop/Data/2019\2019_1\S1A_IW_SLC__1SDV_20190603T165053_20190603T165121_027518_031AEC_3ADD.zip, C:/Users/Prosper/Desktop/Data/2019\2019_1\S1A_IW_SLC__1SDV_20190615T165053_20190615T165121_027693_032036_8012.zip for IW1
Output will be saved as: C:/Users/Prosper/Desktop/Data/2019\processed_vh\S1A_IW_SLC__1SDV_20190603T165053_20190603T165121_027518_031AEC_3ADD_IW1_coh.tif
Reading C:/Users/Prosper/Desktop/Data/2019\2019_1\S1A_IW_SLC__1SDV_20190603T165053_20190603T165121_027518_031AEC_3ADD.zip...
Reading C:/Users/Prosper/Desktop/Data/2019\2019_1\S1A_IW_SLC__1SDV_20190615T165053_20190615T165121_027693_032036_8012.zip...
Processing: C:/Users/Prosper/Desktop/Data/2019\2019_1\S1A_IW_SLC__1SDV_20190603T165053_20190603T165121_027518_031AEC_3ADD.zip, C:/Users/Prosper/Desktop/Data/2019\2019_1\S1A_IW_SLC__1SDV_20190615T165053_20190615T165121_027693_032036_8012.zip for IW2
Output will be saved as: C:/Users/Prosper/Desktop/Data/2019\processed_vh\S1A_IW_SLC__1SDV_20190603T

Processing: C:/Users/Prosper/Desktop/Data/2019\2019_5\S1A_IW_SLC__1SDV_20190605T163448_20190605T163516_027547_031BC8_6437.zip, C:/Users/Prosper/Desktop/Data/2019\2019_5\S1A_IW_SLC__1SDV_20190617T163448_20190617T163516_027722_03210C_CBF8.zip for IW2
Output will be saved as: C:/Users/Prosper/Desktop/Data/2019\processed_vh\S1A_IW_SLC__1SDV_20190605T163448_20190605T163516_027547_031BC8_6437_IW2_coh.tif
Reading C:/Users/Prosper/Desktop/Data/2019\2019_5\S1A_IW_SLC__1SDV_20190605T163448_20190605T163516_027547_031BC8_6437.zip...
Reading C:/Users/Prosper/Desktop/Data/2019\2019_5\S1A_IW_SLC__1SDV_20190617T163448_20190617T163516_027722_03210C_CBF8.zip...
Processing: C:/Users/Prosper/Desktop/Data/2019\2019_5\S1A_IW_SLC__1SDV_20190605T163448_20190605T163516_027547_031BC8_6437.zip, C:/Users/Prosper/Desktop/Data/2019\2019_5\S1A_IW_SLC__1SDV_20190617T163448_20190617T163516_027722_03210C_CBF8.zip for IW3
Output will be saved as: C:/Users/Prosper/Desktop/Data/2019\processed_vh\S1A_IW_SLC__1SDV_20190605T

Processing: C:/Users/Prosper/Desktop/Data/2019\2019_9\S1A_IW_SLC__1SDV_20190610T164315_20190610T164342_027620_031E00_D60E.zip, C:/Users/Prosper/Desktop/Data/2019\2019_9\S1A_IW_SLC__1SDV_20190622T164316_20190622T164343_027795_03233C_F6E6.zip for IW3
Output will be saved as: C:/Users/Prosper/Desktop/Data/2019\processed_vh\S1A_IW_SLC__1SDV_20190610T164315_20190610T164342_027620_031E00_D60E_IW3_coh.tif
Reading C:/Users/Prosper/Desktop/Data/2019\2019_9\S1A_IW_SLC__1SDV_20190610T164315_20190610T164342_027620_031E00_D60E.zip...
Reading C:/Users/Prosper/Desktop/Data/2019\2019_9\S1A_IW_SLC__1SDV_20190622T164316_20190622T164343_027795_03233C_F6E6.zip...
Processing: C:/Users/Prosper/Desktop/Data/2019\2019_10\S1A_IW_SLC__1SDV_20190610T164250_20190610T164318_027620_031E00_96E9.zip, C:/Users/Prosper/Desktop/Data/2019\2019_10\S1A_IW_SLC__1SDV_20190622T164250_20190622T164318_027795_03233C_6CAC.zip for IW1
Output will be saved as: C:/Users/Prosper/Desktop/Data/2019\processed_vh\S1A_IW_SLC__1SDV_2019061